In [ ]:
#this notebook should be used for loading data for running scVelo on slideseq data
# it follows many of the commands used from the scvelo tutorial
# note this was done using scvelo 0.1.12 so some syntax may have changed since this release
# https://scvelo.readthedocs.io/
# the files will be available on single cell portal in the study for slideseqv2

In [ ]:
import numpy as np
%matplotlib inline
import scanpy as sc
import scvelo as scv
scv.logging.print_version()
import pandas as pd

In [ ]:

scv.settings.verbosity = 3  # show errors(0), warnings(1), info(2), hints(3)
scv.settings.set_figure_params('scvelo')  # for beautified visualization

In [ ]:
#####################################
#here we load two matrices for intronic and exonic counts
eyedata_lens_intronic = sc.read_mtx("path/Puck_190926_03_eye_intronic_DGE_lens.mtx")
eyedata_lens_intronic.layers['unspliced'] = eyedata_lens_intronic.X
eyedata_lens_exonic = sc.read_mtx("path/Puck_190926_03_eye_exonic_DGE_lens.mtx")
eyedata_lens_exonic.layers['spliced'] = eyedata_lens_exonic.X

eyedata_lens = scv.utils.merge(eyedata_lens_exonic, eyedata_lens_intronic)

In [ ]:
#here we load precomputed UMAP coordinates, this is due to previously using Seurat for generation of the UMAP coordinates in R
# Seuratv3: https://satijalab.org/seurat/
# Another associated file for generating UMAP coordinates is also included in this repository

#cluster names precomputed in Seurat
eyedata_clusters_lens = scv.load("path/Puck_190926_03_clusters_lens.csv", index_col = 0)
eyedata_lens.obs = eyedata_clusters_lens
#UMAP coordinates for the lens precomputed from seurat
eyedata_UMAP_lens = scv.load("path/Puck_190926_03_umap_lens.csv")
eyedata_lens.obsm["X_umap"] = eyedata_UMAP_lens.values.astype(np.float32)
#spatial coordinates added into scanpy object
eyedata_spatial_lens = scv.load("path/Puck_190926_03_spatial_lens.csv")
eyedata_lens.obsm["X_spatial"] = eyedata_spatial_lens.values.astype(np.float32)
#read in the gene names for the lens
eyedata_lens.var=scv.load("path/Puck_190926_03_vargenes_lens.csv")
eyedata_lens.var.index = eyedata_lens.var["x"]



In [ ]:
#now run normalization
scv.pp.filter_and_normalize(eyedata_lens, min_shared_counts=3)
scv.pp.moments(eyedata_lens, n_pcs=50, n_neighbors=37)
scv.tl.velocity(eyedata_lens)
scv.tl.velocity_graph(eyedata_lens)
scv.pl.velocity_embedding_stream(eyedata_lens, basis='spatial')
scv.pl.velocity_embedding_stream(eyedata_lens, basis = 'umap')
eyedata_lens.var

In [ ]:

scv.tl.recover_dynamics(eyedata_lens, n_top_genes=700, fit_connected_states=True)
scv.tl.velocity(eyedata_lens, mode='stochastic')
scv.tl.velocity_graph(eyedata_lens)
scv.tl.recover_latent_time(eyedata_lens)
#plot onto UMAP coordinates
scv.pl.scatter(eyedata_lens, color='latent_time', fontsize=20, size=100, color_map='gnuplot', colorbar=True, rescale_color=[0,1])


In [ ]:

#plot latent time trajectory over spatial coordinates
scv.pl.scatter(eyedata_lens, color='latent_time', fontsize=20, size = 100, color_map='gnuplot', basis = 'spatial', colorbar=True, rescale_color=[0,1])

In [ ]:
#plot stream plot over spatial coordinates
scv.pl.velocity_embedding_stream(eyedata_lens, basis='spatial')
scv.pl.velocity_embedding_stream(eyedata_lens, basis = 'umap')
scv.pl.scatter(eyedata_lens, color='latent_time', fontsize=20, size = 100, color_map='gnuplot', basis = 'spatial', colorbar=True, rescale_color=[0,1],save="EYE_LATENT_TIME")


In [ ]:
#plot expression of some marker genes from the eye over the latent time axis
scv.pl.scatter(eyedata_lens, x='latent_time', y=['Crybb1','Cryba1','Caprin2','Bfsp2','Prox1'], fontsize=16, size=100,
               n_convolve=None, frameon=False, legend_loc='none', save = "positive_controls_latent_time_eye")

In [ ]:
scv.pl.velocity_embedding(eyedata_lens, basis='spatial', arrow_length=2, arrow_size=1.5, dpi=150)


In [ ]:
#take the coordinates from the scanpy object for plotting individual genes
spatial_bead_coords = pd.DataFrame(eyedata_lens.obsm['X_spatial'], index = eyedata_lens.obs['clusters'].index)

In [ ]:
spatial_bead_coords.columns = ["xcoord","ycoord"]

In [ ]:
####here we want to plot the counts for the spliced and non spliced reads
#load libraries
import pandas as pd
import numpy as np
from IPython.display import display
import seaborn as sns
import matplotlib.pyplot as plt
import sys
import os
from sklearn.preprocessing import StandardScaler
from IPython.display import display
import datetime

now = datetime.datetime.now()
import scipy.optimize
import scipy.stats
import os

%pylab inline

rcParams['axes.spines.right'] = False
rcParams['axes.spines.top'] = False
from sklearn.preprocessing import StandardScaler
import matplotlib.patches as mpatches

import scanpy as sc

In [ ]:
#regeneration of object to just get raw counts not necessary if you have the raw dge saved in the anndata object that was created earlier
eyedata_lens_intronic = sc.read_mtx("path/Puck_190926_03_eye_intronic_DGE_lens.mtx")
eyedata_lens_intronic.layers['unspliced'] = eyedata_lens_intronic.X
eyedata_lens_exonic = sc.read_mtx("path/Puck_190926_03_eye_exonic_DGE_lens.mtx")
eyedata_lens_exonic.layers['spliced'] = eyedata_lens_exonic.X

eyedata_lens_raw = scv.utils.merge(eyedata_lens_exonic, eyedata_lens_intronic)
eyedata_clusters_lens = scv.load("path/Puck_190926_03_clusters_lens.csv", index_col = 0)
eyedata_lens_raw.obs = eyedata_clusters_lens
eyedata_UMAP_lens = scv.load("path/Puck_190926_03_umap_lens.csv")
eyedata_lens_raw.obsm["X_umap"] = eyedata_UMAP_lens.values.astype(np.float32)
eyedata_spatial_lens = scv.load("path/Puck_190926_03_spatial_lens.csv")
eyedata_lens_raw.obsm["X_spatial"] = eyedata_spatial_lens.values.astype(np.float32)
#read in the gene names for the lens
eyedata_lens_raw.var=scv.load("path/Puck_190926_03_vargenes_lens.csv")
eyedata_lens_raw.var.index = eyedata_lens_raw.var["x"]


In [ ]:
eye_exonic_reads_raw = pd.DataFrame(eyedata_lens_raw.layers['spliced'].toarray(), index = eyedata_lens_raw.obs['clusters'].index, columns = eyedata_lens_raw.var.index)

In [ ]:
eye_intronic_reads_raw = pd.DataFrame(eyedata_lens_raw.layers['unspliced'].toarray(), index = eyedata_lens_raw.obs['clusters'].index, columns = eyedata_lens_raw.var.index)

In [ ]:
def plot_one_gene(gene,counts):
    figsize(10, 10)
    pyplot.set_cmap('magma_r')
    plt.scatter(spatial_bead_coords['xcoord']*0.549, spatial_bead_coords['ycoord']*0.549, c=counts[gene], s=15, alpha=1.0)
    plt.axis('equal')
    plt.vmin = 0
    plt.title('{}'.format(gene))
    plt.colorbar();
    #plt.savefig("{}/{}{}.pdf".format("dir,"eye"), dpi = 200)
    #save_result(gene)
    plt.show()
    
interesting_genes = ['Vax2','Aldh1a1','Crybb1','Aldh1a3','Pax6','Six3','Fgfr']

for g in interesting_genes:
    #set intronic or exonic for the dge you wish to use
    plot_one_gene(gene=g,counts=eye_exonic_reads_raw)

